In [1]:
# import everything I need and everything I don't need

import env
import pandas as pd
import pydataset as data
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

# import splitting and imputing functions
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

# import our own acquire module
import acquire

## Big Question:
### Why are Telco customers churning?
### * Find drivers of churn

In [2]:
# Obtain Telco dataset.

telco_df_untouched = acquire.get_telco_data()

In [4]:
telco_df_untouched.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [6]:
# Create function that cleans up the Telco dataset

def prep_telco(df):
        telco_df = df.drop(columns = ['payment_type_id','internet_service_type_id','contract_type_id','customer_id'])
        dummy_df = pd.get_dummies(telco_df[['gender','partner','dependents','phone_service','multiple_lines','online_security','online_backup','device_protection','tech_support','streaming_tv','streaming_movies','paperless_billing','churn','contract_type','internet_service_type','payment_type']],dummy_na=False)
        telco_df = pd.concat([telco_df, dummy_df], axis=1)
        return telco_df

In [9]:
# Clean up the Telco dataset

prep_telco(telco_df_untouched)

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,...,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,Yes,...,0,0,1,1,0,0,0,0,0,1
1,Male,0,No,No,54,No,No phone service,Yes,No,No,...,0,0,1,1,0,0,0,1,0,0
2,Female,0,No,No,56,No,No phone service,Yes,Yes,Yes,...,0,0,1,1,0,0,1,0,0,0
3,Male,0,Yes,Yes,20,No,No phone service,Yes,No,Yes,...,0,0,1,1,0,0,0,1,0,0
4,Male,0,Yes,No,72,Yes,Yes,No,Yes,Yes,...,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,0,Yes,Yes,1,Yes,No,No internet service,No internet service,No internet service,...,1,0,0,0,0,1,0,0,0,1
7039,Female,0,No,No,19,Yes,No,No internet service,No internet service,No internet service,...,1,0,0,0,0,1,0,0,0,1
7040,Female,0,No,No,6,Yes,No,No internet service,No internet service,No internet service,...,1,0,0,0,0,1,0,1,0,0
7041,Male,0,No,No,1,Yes,No,No internet service,No internet service,No internet service,...,1,0,0,0,0,1,0,0,0,1
